In [17]:
import random
from binascii import a2b_hex, b2a_hex
from Crypto.Cipher import AES

In [18]:
# 快速幂取余算法 求(a ^ b) % n
def fast_exp_mod(a, b, n):
    res = 1
    while b != 0:
        if b & 1:
            res = (res * a) % n
        b >>= 1
        a = pow(a, 2, n)
    return res

# 测试
print(fast_exp_mod(2, 10, 11))
print(fast_exp_mod(11, 15, 13))

1
5


In [19]:
# 读文件
def read_file(path):
    with open(path, 'r') as f:
        content = f.read()
    f.close()
    return content

In [20]:
# 写文件
def write_file(content, path):
    with open(path, 'w') as f:
        f.write(content)
    f.close()

In [21]:
# 加密
def RSA_Enc(pk, m): 
    c = fast_exp_mod(m, pk[1], pk[0]) # c = (m ^ e) % N
    return c

In [22]:
# 解密
def RSA_Dec(sk, c): 
    m = fast_exp_mod(c, sk[1], sk[0]) # m = (c ^ d) % N
    return m

In [23]:
# WUP类
class WUP:
    def __init__(self, request="", key=""):
        # AES加密的 WUP Request
        self.aes_en_wup = request
        # RSA加密的 AES密钥
        self.rsa_en_aes_key = key 

In [24]:
# Client类
class Client:
    def __init__(self):
        # RSA公钥
        self.rsa_pk = eval(read_file('RSA_Public_Key.txt'))
        # 生成AES密钥
        self.aes_key = random.randrange(2 ** 127, 2 ** 128)
        write_file(hex(self.aes_key), 'AES_Key.txt')
    
    def send_message(self, msg):
        # WUP Request写入文件
        write_file(hex(int(msg.encode('utf-8').hex(), 16)), 'WUP_Request.txt')
        # 为AES加密补齐位数
        while len(msg) % 16 != 0:
            msg += '\0'

        # AES加密 WUP Request
        AES_Cryptor = AES.new(a2b_hex(hex(self.aes_key)[2:]), AES.MODE_ECB)
        aes_en_wup = b2a_hex(AES_Cryptor.encrypt(msg.encode('utf-8')))
        
        # RSA加密 AES密钥
        rsa_en_aes_key = RSA_Enc(self.rsa_pk, self.aes_key)
        
        # AES加密的 WUP Request写入文件
        write_file(hex(int(aes_en_wup.hex(), 16)), "AES_Encrypted_WUP.txt")
        # History Message（包括AES加密的 WUP Request和RSA加密的 AES密钥）写入文件
        write_file(hex(int(aes_en_wup.hex(), 16)) + '\n' + hex(rsa_en_aes_key), 'History_Message.txt')

        # 发送WUP报文
        w = WUP(aes_en_wup, rsa_en_aes_key)
        return w

In [25]:
class Server:
    def __init__(self):
        # RSA公钥
        self.rsa_pk = eval(read_file('RSA_Public_Key.txt'))
        # RSA私钥
        self.rsa_sk = eval(read_file('RSA_Secret_Key.txt'))
        # AES密钥，初始尚未收到
        self.aes_key = 0

    def receive_message(self, w):
        # RSA私钥解密 WUP报文中的 AES密钥
        aes_key = bin(RSA_Dec(self.rsa_sk, w.rsa_en_aes_key))[-128:]
        aes_key = int(aes_key, 2)

        # 补齐位数
        aes_key_string = ""
        for i in hex(aes_key)[2:]:
            aes_key_string += i
        while len(aes_key_string) < 32:
            aes_key_string = "0" + aes_key_string
        self.aes_key = a2b_hex(aes_key_string)
        
        # AES密钥解密 WUP报文中的 WUP Request
        AES_Decrypter = AES.new(self.aes_key, AES.MODE_ECB)
        plain_message = b2a_hex(AES_Decrypter.decrypt(a2b_hex(w.aes_en_wup)))
        
        return plain_message

In [26]:
# 随机种子
random.seed(520030910281)

# 初始化
client = Client()
server = Server()

# WUP Request
request = "Sample Request"
# Client发送WUP获得历史消息
history_message = client.send_message(request)

# Server收到消息，返回解密的WUP Request
print(str(a2b_hex(server.receive_message(history_message)), encoding='utf-8'))

Sample Request  


In [27]:
# CCA2攻击过程
def CCA2_Attack(server, history_msg):
    # 已知RSA公钥
    rsa_pk = eval(read_file('RSA_Public_Key.txt'))

    # 要猜测的AES密钥，128-bit
    current_key = 0
    # 迭代128次
    for ite in range(128, 0, -1):
        # 本次猜测密钥
        test_key = int(current_key >> 1) + (1 << 127)

        # 填充用 WUP Request
        pad_request = 'whatever'
        while len(pad_request) % 16 != 0:
            pad_request += '\0'

        # 用猜测的AES密钥加密 WUP Request
        AES_Cryptor = AES.new(a2b_hex(hex(test_key)[2:]), AES.MODE_ECB)
        aes_en_wup = str(b2a_hex(AES_Cryptor.encrypt(pad_request.encode('utf-8'))), encoding='utf-8')

        # 用C（RSA公钥加密后的AES密钥）和 RSA公钥 计算 C_b
        factor = fast_exp_mod(2, (ite - 1) * rsa_pk[1], rsa_pk[0])
        rsa_en_aes_key = fast_exp_mod(history_msg.rsa_en_aes_key * factor, 1, rsa_pk[0])

        # 构造WUP发送给Server，得到回复
        w = WUP(aes_en_wup, rsa_en_aes_key)
        response = server.receive_message(w)

        # 如果WUP Request解密成功说明该位猜测正确，否则取反
        if response == b2a_hex(bytes(pad_request, encoding='utf-8')):
            current_key = test_key
        else:
            test_key = int(current_key >> 1)
            current_key = test_key
    
    # 用最终猜得的AES密钥解密History Message中的WUP Request
    AES_Decrypter = AES.new(a2b_hex(hex(current_key)[2:]), AES.MODE_ECB)
    plain_msg = str(AES_Decrypter.decrypt(a2b_hex(history_message.aes_en_wup)), encoding='utf-8')
    plain_msg = plain_msg.rstrip('\0')

    # 验证，与client最初生成的AES密钥一致则攻击成功
    if current_key == client.aes_key:
        print("Attack Success!")
        print('History Message Request:', plain_msg)
        print('AES Key:', hex(current_key))
    else:
        print("Attack Fail!")
        print('Guessed AES Key:', current_key)
    
# 实施攻击
CCA2_Attack(server, history_message)

Attack Success!
History Message Request: Sample Request
AES Key: 0xb65160645fd3d71e7f21aed0e7d08e0f
